In [10]:
import os

#Inserire il path del set di dati dove si trovano i comuni
path = "C:/Users/leonn/Documents/Progetti/TesiUnicam/Data/"

#del path ricava una lista di tutti i file e directory
listComuni = os.listdir(path)

#rimuovere dalla lista i file (sono quelli contrassegnati da una estensione ".")
for comune in listComuni:
    if "." in comune or "Output" in comune or "Plot" in comune:
        listComuni.remove(comune)

In [11]:
#il seguente script stampa tutti i path per arrivare ai file "Istanze.csv"
for comune in listComuni:
    comune = path+comune+"/Istanze.csv"
    print(comune)

C:/Users/leonn/Documents/Progetti/TesiUnicam/Data/Udine/Istanze.csv


In [12]:
#if not os.path.exists(path+"/Output")
#os.mkdir(path+"/Output")
for comune in listComuni:
    os.mkdir(path+"/Output/"+comune)

In [13]:
import pandas
import csv

def renamedLabel(df):
    vec = []
    fChar = ord('a')
    df_sort = df.sort_values('activity')
    precEl = df_sort.at[0, 'activity']
    #attraversamento dataframe per rinonimare
    for el in df_sort['activity']:
        if(el == precEl):
            vec.append(chr(fChar))
        else:
            precEl = el
            fChar += 1
            vec.append(chr(fChar))
    df_sort.insert(df_sort.shape[1], "renamed", vec)
    return df_sort.sort_index()

for comune in listComuni:
    print("Sto processando il comune "+comune+"...")
    #blocco 1

    df1 = pandas.read_csv(path+comune+"/" + "Istanze.csv", encoding='ISO-8859-1', low_memory='false')

    df1_droped=df1.drop(['id passo', 'id iter', 'versione iter','desc iter',  'flg iter default', 'area iter', 'id istanza padre', 'id flusso', 'id flusso padre',
                        'id flusso istanza padre', 'id istanza padre', 'id stato', 'ordine generazione', 'stato inizio ms', 'stato fine ms','utente super user','utente internal user' ], axis=1 )
    
    df1_droped = df1_droped.fillna('')

    #blocco 2
    df1_droped['activity'] = df1_droped['tipo stato'].str.cat(df1_droped['desc stato'],sep='_')

    #blocco 3 (quello con piu necessità computazionale)
    # per ottimizzare usare il parallelismo o eventuali librerie di set/map
    labels=df1_droped['activity'] 
    names = {}
    for l in labels:
        names.setdefault(l, len(names)+1)
        df1_droped['coded activity'] = df1_droped['activity'].map(names)

    #blocco 4 (richiede blocco3)
    vec = []
    for i in df1_droped['coded activity'] :
        i=int(i)
        i=i+96
        vec.append(chr(i))
    df1_droped.insert(9, "renamed", vec)

    #df1_droped = renamedLabel(df1_droped)

    #blocco 5
    group = df1_droped.groupby('nome iter')  
    for k, gr in group:
        # do your stuff instead of print
        #print(k)
        #print(type(gr)) # This will output <class 'pandas.core.frame.DataFrame'>
        #print(gr)
        # You can save each 'gr' in a csv as follows
        gr.to_csv(path+'/Output/'+comune+'/{}.csv'.format(k))
    print(comune+" processato!\n")

Sto processando il comune Udine...
Udine processato!

